<a href="https://colab.research.google.com/github/brilliantfriezka/Robotics-and-Intelligent-Systems/blob/main/UTS/Camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install symforce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 82 kB 195 kB/s 
     |████████████████████████████████| 1.5 MB 45.0 MB/s 
     |████████████████████████████████| 6.5 MB 19.6 MB/s 
     |████████████████████████████████| 1.5 MB 40.7 MB/s 
     |████████████████████████████████| 70 kB 4.6 MB/s 
     |████████████████████████████████| 49 kB 4.0 MB/s 
     |████████████████████████████████| 96 kB 3.4 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting uninstall: sympy
    Found existing installation: sympy 1.7.1
    Uninstalling sympy-1.7.1:
      Successfully uninstalled sympy-1.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

In [2]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
import symforce.symbolic as sf

In [3]:
# Membuat objek kalibrasi kamera linier:
linear_camera_cal = sf.LinearCameraCal.symbolic("cal")
display(linear_camera_cal)

<LinearCameraCal
  focal_length=[cal.f_x, cal.f_y],
  principal_point=[cal.c_x, cal.c_y],
  distortion_coeffs=[]>

In [4]:
# Deproyeksi titik-titik yang tertulis di bingkai kamera sebagai berikut:
camera_point = sf.V3.symbolic("p")
camera_ray, _ = linear_camera_cal.camera_ray_from_pixel(camera_point)
display(camera_ray)

⎡-cal.cₓ + p₀ ⎤
⎢──────────── ⎥
⎢   cal.fₓ    ⎥
⎢             ⎥
⎢-cal.c_y + p₁⎥
⎢─────────────⎥
⎢   cal.f_y   ⎥
⎢             ⎥
⎣      1      ⎦

In [5]:
camera_point_reprojected, _ = linear_camera_cal.pixel_from_camera_point(
    camera_ray,
)
display(camera_point_reprojected)

⎡p₀⎤
⎢  ⎥
⎣p₁⎦

In [6]:
# Menggunakan objek kalibrasi kamera, membuat kamera dengan parameter tambahan (seperti ukuran gambar):
linear_camera = sf.Camera(
    calibration=sf.LinearCameraCal(
        focal_length=(440, 400),
        principal_point=(320, 240),
    ),
    image_size=(640, 480),
)
display(linear_camera)

<Camera
  CameraCal=<LinearCameraCal
  focal_length=[440, 400],
  principal_point=[320, 240],
  distortion_coeffs=[]>
  image_size=[640, 480]>

In [7]:
point_in_FOV = sf.V3(0, 0, 1)
point_outside_FOV = sf.V3(100, 0, 1)
for point in (point_in_FOV, point_outside_FOV):
    pixel, is_valid = linear_camera.pixel_from_camera_point(point)
    print(
        "point={} -> pixel={}, is_valid={}".format(
            point.to_storage(),
            pixel.to_storage(),
            is_valid,
        )
    )

point=[0, 0, 1] -> pixel=[320, 240], is_valid=1
point=[100, 0, 1] -> pixel=[44320, 240], is_valid=0


In [8]:
# Membuat kamera dengan pose tertentu:
linear_posed_camera = sf.PosedCamera(
    pose=sf.Pose3(
        # Memutar kamera 180 derajat pada sumbu y
        R=sf.Rot3.from_yaw_pitch_roll(0, sf.pi, 0),
        t=sf.V3(),
    ),
    calibration=linear_camera.calibration,
    image_size=(640, 480),
)
display(linear_posed_camera)

<PosedCamera
  Pose=<Pose3 R=<Rot3 <Q xyzw=[0, 1, 0, 0]>>, t=(0, 0, 0)>
  Camera=<PosedCamera
  CameraCal=<LinearCameraCal
  focal_length=[440, 400],
  principal_point=[320, 240],
  distortion_coeffs=[]>
  image_size=[640, 480]>>

In [10]:
# Memberikan pose yang dapat digunakan untuk mengubah titik antara bingkai global dan bingkai gambar:
global_point = sf.V3(0, 0, -1)
print(
    "point in global coordinates={} (in camera coordinates={})".format(
        global_point.to_storage(),
        (linear_posed_camera.pose * global_point).to_storage(),
    )
)

pixel, is_valid = linear_posed_camera.pixel_from_global_point(global_point)
print(
    "global_point={} -> pixel={}, is_valid={}".format(
        global_point.to_storage(), pixel.to_storage(), is_valid
    )
)

point in global coordinates=[0, 0, -1] (in camera coordinates=[0, 0, 1])
global_point=[0, 0, -1] -> pixel=[320, 240], is_valid=1


In [11]:
# Mengubah titik dalam koordinat piksel kembali ke bingkai global (diberi rentang):
range_to_point = (global_point - linear_posed_camera.pose.t).norm()
global_point_reprojected, is_valid = linear_posed_camera.global_point_from_pixel(
    pixel, range_to_point=range_to_point
)
display(global_point_reprojected)

⎡ 0  ⎤
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎣-1.0⎦

In [12]:
# Merubah sudut kecil pada gulungan perturb kamera kedua sedikit dari yang pertama
perturbed_rotation = linear_posed_camera.pose.R * sf.Rot3.from_yaw_pitch_roll(0, 0, 0.5)
target_posed_cam = sf.PosedCamera(
    pose=sf.Pose3(R=perturbed_rotation, t=sf.V3()),
    calibration=linear_camera.calibration,
)
# Memberikan Warp pixel dari kamera sumber ke kamera target dengan rentang terbalik
target_pixel, is_valid = linear_posed_camera.warp_pixel(
    pixel=sf.V2(320, 240),
    inverse_range=1.0,
    target_cam=target_posed_cam,
)
display(target_pixel)

⎡      320       ⎤
⎢                ⎥
⎣458.520995937516⎦

In [13]:
# Menggunakan kalibrasi linier, tetapi dapat menggunakan jenis kalibrasi lain juga seperti:
atan_cam = sf.ATANCameraCal(
    focal_length=[380.0, 380.0],
    principal_point=[320.0, 240.0],
    omega=0.35,
)
camera_ray, is_valid = atan_cam.camera_ray_from_pixel(sf.V2(50.0, 50.0))
display(camera_ray)
pixel, is_valid = atan_cam.pixel_from_camera_point(camera_ray)
display(pixel)

⎡-0.72576759882138 ⎤
⎢                  ⎥
⎢-0.510725347318749⎥
⎢                  ⎥
⎣        1         ⎦

⎡49.9999999999999⎤
⎢                ⎥
⎣      50.0      ⎦